In [1]:
#written by Noah Friedman (a template for scripts to be excuted in the spyder environment
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'

sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util

In [12]:
#returns the name of the gene and left most position spanning the msi indel
#uses re stuff
def get_left_aligned_allele_name(hgvsNames):
    
    positions = []
    geneName = hgvsNames[0].split('_p.')[0]
    for entry in hgvsNames:
        if len(entry.split('_p.')) == 2: #ignore weirdly formatted hgvs names
            variantNotation = entry.split('_p.')[1]
            number = variantNotation[1:]
            refAA = variantNotation[0]
            position = re.match('\d*', number).group(0)
            positions.append((position, refAA))
    
    if len(positions) == 0: return None #if all the hgvs names were ill formatted return None
    
    minEntry = sorted(positions)[0] #this is a sorted list of tuples the first thing is the position second is the reference aa
    return geneName + '_p.' + str(minEntry[1]) + str(minEntry[0])

#collapses all indels within 1 bp of each other for a start to be at the same location/name for matching
def standardize_allele_names(msiLengthInfo, observedMuts):
    
    neverObservedSites = set([]) #all the names of sites from criags msi file we cant match with the real maf
    msiSitesToNameMapping = {} #a dictionary mapping each msi site allele from craigs file to its corrected name
    mafMsiSiteToNameMapping = {} #a dictionary mapping each msi site allele from the maf to its corrected name
    
    cntr = 0.0
    for hgvs in set(msiLengthInfo['allele']):
        
        cntr += 1
        if cntr%500 == 0: print 100*(cntr/len(set(msiLengthInfo['allele']))), 'percent done'
        
        startPos = msiLengthInfo[msiLengthInfo['allele'] == hgvs]['Start_Position']
        
        
        if startPos.shape[0] == 1:
            #we want all names given to indels near (within 1position) of the start position of the MSI site in Craig's file
            putativeVariantNames = list(set(observedMuts[(abs(observedMuts['Start_Position'] - int(startPos)) < 2)
                                                    & (observedMuts['Variant_Type'].isin(set(['INS', 'DEL'])))]['allele']))
            
            if len(putativeVariantNames) == 0:
                neverObservedSites.add(hgvs) #if it cant be matched in the MAF we add it to never observed sites
                #note some of there are likely to be actually matched but missed by my method
            else:
                trueVariantName = get_left_aligned_allele_name(putativeVariantNames)
                
                #NOW WE PROPERLY create the mappings
                for putativeVariantName in putativeVariantNames:
                    mafMsiSiteToNameMapping[putativeVariantName] = trueVariantName
                msiSitesToNameMapping[hgvs] = trueVariantName
        else:
            pass #ignore variants with multiple start position in the msi info file
        
    return neverObservedSites, msiSitesToNameMapping, mafMsiSiteToNameMapping

In [2]:
msiSummary = pd.read_table('/Users/friedman/Desktop/impact_microsatellites.txt')


/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0,76,97) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
msiSummary['repeat_times_corrected'] = msiSummary['repeat_times'].apply(lambda x: x if x < 9 else None)
msiSummary['basePairClass'] = msiSummary['Tumor_Seq_Allele2'].apply(lambda x:
                                                                   'AT' if x in ['A', 'T']
                                                                   else 'CG' if x in ['C', 'G']
                                                                   else 'Other')

In [18]:
tumorSuppressors = set(['ERRFI1', 'ASXL2', 'PMAIP1', 'ACTG1', 'SUFU', 'FBXO11', 'MEN1', 'FAM58A', 'B2M', 'RB1', 'DUSP22', 'SESN1', 'GPS2', 'RAD51D', 'SMG1', 'CDC73', 'MAP3K1', 'SMARCB1', 'INPP4B', 'PARK2', 'SMAD4', 'CBFB', 'CDH1', 'PPP6C', 'SETDB1', 'SETDB2', 'NF2', 'CDKN2B', 'CDKN2C', 'CDKN2A', 'DDX3X', 'PIK3R1', 'BARD1', 'PDS5B', 'KLF4', 'SPRED1', 'VHL', 'SMAD2', 'PMS1', 'PMS2', 'SETD2', 'GATA3', 'TBL1XR1', 'MUTYH', 'SOCS1', 'FAM175A', 'ROBO1', 'ARID1B', 'ARID1A', 'TCF7L2', 'STK11', 'FOXA1', 'PTEN', 'FAT1', 'FAS', 'CYLD', 'MAX', 'SH2D1A', 'APC', 'NTHL1', 'CTCF', 'KDM5C', 'KMT2C', 'ZFHX3', 'FOXP1', 'PIGA', 'CDKN1B', 'CDKN1A', 'FUBP1', 'MSH2', 'ID3', 'TNFRSF14', 'TRAF3', 'EP400', 'BRIP1', 'ARID4A', 'ARID4B', 'XRCC2', 'DAXX', 'SDHAF2', 'ASXL1', 'AMER1', 'RASA1', 'EGR1', 'MST1', 'SOX17', 'RUNX1', 'PIK3R3', 'NCOR1', 'NF1', 'JAK1', 'PTPRD', 'CHEK2', 'CHEK1', 'SMC1A', 'TMEM127', 'STAG1', 'RAD51', 'TCF3', 'STAG2', 'ARID2', 'RAD50', 'RNF43', 'PARP1', 'BLM', 'CUX1', 'RECQL', 'RAD21', 'PTPN2', 'PTPN1', 'SLX4', 'INHA', 'PAX5', 'IRF1', 'TP53', 'HLA-A', 'IRF8', 'CBL', 'TOP1', 'SHQ1', 'PRDM1', 'NSD1', 'ATXN2', 'CREBBP', 'HDAC4', 'SESN2', 'PPP2R1A', 'EPHA7', 'ATM', 'EPHA3', 'POT1', 'SMAD3', 'MOB3B', 'TBX3', 'POLE', 'ATR', 'FANCD2', 'FH', 'BCORL1', 'SOX9', 'IKZF3', 'TSC1', 'TP63', 'MRE11A', 'SDHC', 'BTG1', 'POLD1', 'CIITA', 'SMC3', 'SAMHD1', 'RTEL1', 'ECT2L', 'PIK3R2', 'CRBN', 'FANCC', 'NBN', 'FANCA', 'HLA-B', 'RECQL4', 'DUSP4', 'ERCC2', 'FBXW7', 'TGFBR2', 'TGFBR1', 'MSH3', 'RBM15', 'TET1', 'TET3', 'SESN3', 'MGA', 'LTB', 'FOXL2', 'SH2B3', 'BCOR', 'HIST1H1D', 'ATRX', 'EP300', 'RAD51C', 'RAD51B', 'HIST1H1B', 'TNFAIP3', 'DICER1', 'ARID5B', 'LATS2', 'FOXO1', 'KEAP1', 'EZH2', 'SP140', 'NKX3-1', 'PBRM1', 'PALB2', 'CIC', 'BRCA1', 'DTX1', 'FLCN', 'SPEN', 'CD58', 'ERCC3', 'ERCC4', 'MSH6', 'BCL11B', 'BMPR1A', 'ERF', 'BRCA2', 'NOTCH2', 'EED', 'MITF', 'ELF3', 'SMARCA4', 'BBC3', 'ANKRD11', 'CEBPA', 'BCL2L11', 'AXIN2', 'AXIN1', 'CDK12', 'ESCO2', 'MLH1', 'SDHB', 'MED12', 'HNF1A', 'RYBP', 'ATP6V1B2', 'DNMT3B', 'KMT2B', 'KMT2A', 'DNMT3A', 'NFKBIA', 'TRAF5', 'KMT2D', 'SPOP', 'RBM10', 'P2RY8', 'TP53BP1', 'TSC2', 'KDM6A', 'EPCAM', 'PHOX2B', 'NPM1', 'BCL10', 'LATS1', 'HOXB13', 'ARID3A', 'PTPRT', 'PTPRS', 'INPPL1', 'NOTCH4', 'TET2', 'NOTCH1', 'CASP8', 'NOTCH3', 'GRIN2A', 'MAP2K4', 'WT1', 'BACH2', 'SDHA', 'BAP1', 'PTCH1', 'SDHD'])
msiSummary['isTSG'] = msiSummary['Hugo_Symbol'].apply(lambda x: 1 if x in tumorSuppressors else 0)

In [68]:
def assign_indel_mut_ratings(msiSum):
    msiDict = {}
    for i in range(5,9):
        for val in ['AT', 'CG']:
            mutType = val + '_' + str(i) + '.0'
            msiDict['tsgFrac_' + mutType] = np.nanmean(msiSum[(msiSum['basePairClass'] == val) & (msiSum['repeat_times_corrected'] == i)]['isTSG'])
            msiDict['oncogeneFrac_' + mutType] = 1 - np.nanmean(msiSum[(msiSum['basePairClass'] == val) & (msiSum['repeat_times_corrected'] == i)]['isTSG'])
    return msiDict
        

In [ ]:
indelMutRankings = assign_indel_mut_ratings(msiSummary)

In [ ]:
allImpactMuts = analysis_utils.load_in_df_with_progress(filePath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf', nLinesFile = 275000)


In [ ]:
allImpactMuts = analysis_utils.map_cases_to_msi_sensor_class(allImpactMuts, msiSensorInfo=pathPrefix + '/ifs/work/taylorlab/friedman/mskImpactAsOfMarch2019/dmp/mskimpact/data_clinical_sample.txt')


In [19]:
allImpactMuts['allele'] = allImpactMuts.apply(lambda row: str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']), axis=1)
msiSummary['allele'] = msiSummary.apply(lambda row: str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']), axis=1)

In [ ]:
neverObservedSites, msiSitesToNameMapping, mafMsiSiteToNameMapping =  standardize_allele_names(msiSummary, allImpactMuts)

allImpactMuts['correctedAllele'] = allImpactMuts['allele'].apply(lambda x:
                                                                 mafMsiSiteToNameMapping[x] if x in mafMsiSiteToNameMapping else None)
msiSummary['correctedAllele'] = msiSummary['allele'].apply(lambda x:
                                                           msiSitesToNameMapping[x] if x in msiSitesToNameMapping else None)

In [51]:
basePairClassDict = dict(zip(msiSummary['correctedAllele'], msiSummary['basePairClass']))
repeatTimesDict = dict(zip(msiSummary['correctedAllele'], msiSummary['repeat_times_corrected']))
allImpactMuts['basePairAndClass'] = allImpactMuts['correctedAllele'].apply(lambda x:
                None if x not in basePairClassDict
                else None if x not in repeatTimesDict
                else basePairClassDict[x] + '_' + str(repeatTimesDict[x]))

In [52]:
msiMuts = allImpactMuts[allImpactMuts['caseMsiClass'] == 'Instable']

In [61]:
msiMutsRepeatsLimited = msiMuts[~msiMuts['basePairAndClass'].isin(['Other_3.0', 'Other_4.0', 'Other_5.0', 
                                                                   'Other_6.0', 'Other_7.0', 'Other_8.0',
                                                                   'Other_nan', 'AT_nan', 'CG_nan'])]


In [82]:
listOfDicts = []
for case in set(msiMutsRepeatsLimited['Tumor_Sample_Barcode']):
    caseMafIndels = msiMutsRepeatsLimited[(msiMutsRepeatsLimited['Tumor_Sample_Barcode'] == case) & (msiMutsRepeatsLimited['Variant_Type'].isin(['INS', 'DEL']))]
    tsgMaf = caseMafIndels[caseMafIndels['Hugo_Symbol'].isin(tumorSuppressors)]
    oncogeneMaf = caseMafIndels[~caseMafIndels['Hugo_Symbol'].isin(tumorSuppressors)]
    
    nTsgExpected = 0
    nOncogeneExpected = 0
    for mutType, count in Counter(caseMafIndels['basePairAndClass']).items():
        nOncogeneExpected += count*indelMutRankings['oncogeneFrac_' + mutType]
        nTsgExpected += count*indelMutRankings['tsgFrac_' + mutType]
    nTsg = tsgMaf.shape[0]
    nOncogene = oncogeneMaf.shape[0]
    nIndels = caseMafIndels.shape[0]
    
    listOfDicts.append({'Tumor_Sample_Barcode': case, 'nOncogene': nOncogene, 'nTsg': nTsg,
                       'nTsgExpected': nTsgExpected, 'nOncogeneExpected': nOncogeneExpected, 'nIndels': nIndels})

df = pd.DataFrame(listOfDicts)
        

In [83]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/msiObsVsExpected.tsv', index=False, sep='\t')

In [84]:
impactSigs = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/signatures_from_unfiltered_maf.txt')
impactSigs = mutationSigUtils.merge_signature_columns(impactSigs)
impactSigs['dominantSignature'] = impactSigs.apply(lambda row: 
        mutationSigUtils.get_dominant_signature(row.to_dict(), cols=None, prefix='mean', notEnoughMuts= True), axis=1)
